In [1]:
.libPaths(c("/home/data/t060324/R/x86_64-pc-linux-gnu-library/4.1","/home/data/refdir/Rlib"))
library(scibet)
library(Seurat)
library(scater)
library(scran)
library(dplyr)
library(Matrix)
library(cowplot)
library(ggplot2)
library(harmony)
setwd('/home/data/t060324/jobs/aimin/')

Registered S3 method overwritten by 'spatstat.core':
  method          from
  formula.glmmPQL MASS

Attaching SeuratObject

Attaching sp

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumpro

In [3]:
sc_data <- readRDS(file='./output/1.sangleCell_pre/OV_GSE165897_final.RDS')

In [84]:
library(CARD)

Registered S3 methods overwritten by 'registry':
  method               from 
  print.registry_field proxy
  print.registry_entry proxy

Warning message:
“replacing previous import ‘RcppML::nmf’ by ‘NMF::nmf’ when loading ‘CARD’”


In [85]:
### scRNA count
sc_count <- GetAssayData(sc_data,assay = 'RNA',slot = 'counts')
sc_count[1:6,1:6]

6 x 6 sparse Matrix of class "dgCMatrix"
              AAACGGGCACTCTGTC-EOC443_iOme1_1 AAACGGGTCCAAACTG-EOC443_iOme1_1
RP11-34P13.7                                .                               .
RP11-34P13.8                                .                               .
CICP27                                      .                               .
RP11-34P13.15                               .                               .
RP11-34P13.13                               .                               .
FO538757.1                                  .                               .
              AAAGATGGTCCATGAT-EOC443_iOme1_1 AAAGTAGGTGGCGAAT-EOC443_iOme1_1
RP11-34P13.7                                .                               .
RP11-34P13.8                                .                               .
CICP27                                      .                               .
RP11-34P13.15                               .                               .
RP11-34P13.13          

In [86]:
### sc mate data
sc_data$cellID <- rownames(sc_data@meta.data)
sc_meta <- sc_data@meta.data[,c('cellID','cellType_auto','patient_id')]
head(sc_meta)

,cellID,cellType_auto,patient_id
,<chr>,<chr>,<chr>
AAACGGGCACTCTGTC-EOC443_iOme1_1,AAACGGGCACTCTGTC-EOC443_iOme1_1,B,EOC443
AAACGGGTCCAAACTG-EOC443_iOme1_1,AAACGGGTCCAAACTG-EOC443_iOme1_1,B,EOC443
AAAGATGGTCCATGAT-EOC443_iOme1_1,AAAGATGGTCCATGAT-EOC443_iOme1_1,B,EOC443
AAAGTAGGTGGCGAAT-EOC443_iOme1_1,AAAGTAGGTGGCGAAT-EOC443_iOme1_1,B,EOC443
AAAGTAGTCAGAGGTG-EOC443_iOme1_1,AAAGTAGTCAGAGGTG-EOC443_iOme1_1,plasma,EOC443
AACCATGAGTGTCCCG-EOC443_iOme1_1,AACCATGAGTGTCCCG-EOC443_iOme1_1,plasma,EOC443


In [87]:
sample <- list.files(path = './data/GSE211956_RAW/',pattern = '_ST.RDS')
names(sample) <- substr(sample,1,10)
sample

GSM6506110          GSM6506111          GSM6506112          GSM6506113 
"GSM6506110_ST.RDS" "GSM6506111_ST.RDS" "GSM6506112_ST.RDS" "GSM6506113_ST.RDS" 
         GSM6506114          GSM6506115          GSM6506116          GSM6506117 
"GSM6506114_ST.RDS" "GSM6506115_ST.RDS" "GSM6506116_ST.RDS" "GSM6506117_ST.RDS"

In [88]:
for(samplefile in names(sample)){
ST_obj <- readRDS(file = paste0('./data/GSE211956_RAW/',sample[samplefile]))
spatial_count <- GetAssayData(ST_obj,assay = 'Spatial',slot = 'counts')
spatial_location <- ST_obj@images$image@coordinates[,c('row','col')]
colnames(spatial_location) <- c('x','y')
CARD_obj = createCARDObject(
	sc_count = sc_count,
	sc_meta = sc_meta,
	spatial_count = spatial_count,
	spatial_location = spatial_location,
	ct.varname = "cellType_auto",
	ct.select = unique(sc_meta$cellType_auto),
	sample.varname = "patient_id",
	minCountGene = 100,
	minCountSpot = 5) 
CARD_obj = CARD_deconvolution(CARD_object = CARD_obj)
tmp <- CARD_obj@Proportion_CARD[match(rownames(ST_obj@meta.data),rownames(CARD_obj@Proportion_CARD)),]
rownames(tmp) <- rownames(ST_obj@meta.data)
ST_obj@meta.data <- cbind(ST_obj@meta.data,tmp)
saveRDS(ST_obj,file = paste0('./data/GSE211956_RAW/',sample[samplefile]))
saveRDS(CARD_obj,file = paste0('./data/GSE211956_RAW/',samplefile,'_ST_CARD.RDS'))
}

## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...
## create reference matrix from scRNASeq...


Loading required package: nnls


Attaching package: ‘wrMisc’


The following objects are masked from ‘package:MatrixGenerics’:

    colSds, rowSds


The following objects are masked from ‘package:matrixStats’:

    colSds, rowSds




## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...
## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...
## create reference matrix from scRNASeq...
## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...
## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...
## create reference matrix from scRNASeq...
## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...
## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...
## create reference matrix from scRNASeq...
## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...
## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...
## create reference matrix from scRNASeq...
## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...
## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...
## create